# bob4

### STATUS

#### create_dataset_fcn8
- Basically able to return a fuzzy version of the input
- Not sure how much more I can tweak the model to reach the next time stamp

#### create_dataset_diff
- Currently doing **validation data** on both training and evaluation!!
- Trying to overfit and show validation function works, too

In [1]:
import os
import numpy as np
import tensorflow as wtf
import data_utils as du

/home/noodles/.conda/envs/turntable/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
BATCH_SIZE = 4
EPOCHS = 1

num_train, num_val, num_test = 70, 10, 20
train_data,val_data,test_data = du.create_dicts(num_train, num_val, num_test)

In [3]:
MODEL_NAME = 'diff-001'

In [4]:
# Attempting that FCN8-based model, t->t+1

def model_fn(features,
             labels,
             mode,
             params):
    with wtf.device('/device:GPU:0'):
        inputs = features
        
        conv1_1 = wtf.layers.conv2d(inputs,
                                    filters=64,
                                    kernel_size=3,
                                    strides=1,
                                    padding='same',
                                    activation=wtf.nn.relu,
                                    kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                    name='conv1_1')
        conv1_2 = wtf.layers.conv2d(conv1_1,
                                    filters=64,
                                    kernel_size=3,
                                    strides=1,
                                    padding='same',
                                    activation=wtf.nn.relu,
                                    kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                    name='conv1_2')
        pool1 = wtf.layers.max_pooling2d(conv1_2,
                                         pool_size=2,
                                         strides=2,
                                         padding='same',
                                         name='pool1')
        
        conv2_1 = wtf.layers.conv2d(pool1,
                                    filters=256,
                                    kernel_size=3,
                                    strides=1,
                                    padding='same',
                                    activation=wtf.nn.relu,
                                    kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                    name='conv2_1')
        conv2_2 = wtf.layers.conv2d(conv2_1,
                                    filters=256,
                                    kernel_size=3,
                                    strides=1,
                                    padding='same',
                                    activation=wtf.nn.relu,
                                    kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                    name='conv2_2')
        pool2 = wtf.layers.max_pooling2d(conv2_2,
                                         pool_size=2,
                                         strides=2,
                                         padding='same',
                                         name='pool2')
        
        conv3_1 = wtf.layers.conv2d(pool2,
                                    filters=1024,
                                    kernel_size=3,
                                    strides=1,
                                    padding='same',
                                    activation=wtf.nn.relu,
                                    kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                    name='conv3_1')
        conv3_2 = wtf.layers.conv2d(conv3_1,
                                    filters=1024,
                                    kernel_size=3,
                                    strides=1,
                                    padding='same',
                                    activation=wtf.nn.relu,
                                    kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                    name='conv3_2')
        pool3 = wtf.layers.max_pooling2d(conv3_2,
                                         pool_size=2,
                                         strides=2,
                                         padding='same',
                                         name='pool3')
        
        conv4_1 = wtf.layers.conv2d(pool3,
                                    filters=2048,
                                    kernel_size=1,
                                    strides=1,
                                    padding='same',
                                    activation=wtf.nn.relu,
                                    kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                    name='conv4_1')
        
        conv4_2 = wtf.layers.conv2d(conv4_1,
                                    filters=2048,
                                    kernel_size=1,
                                    strides=1,
                                    padding='same',
                                    activation=wtf.nn.relu,
                                    kernel_initializer=wtf.variance_scaling_initializer(scale=2.0),
                                    name='conv4_2')
        
        dout = wtf.layers.dropout(conv4_2,
                                rate=0.0)
        
        deconv1 = wtf.layers.conv2d_transpose(dout,
                                              filters=pool2.get_shape().as_list()[-1],
                                              kernel_size=4,
                                              strides=2,
                                              padding='same')
        
        deconv1_skip = wtf.add(deconv1, pool2)
        
        deconv2 = wtf.layers.conv2d_transpose(deconv1_skip,
                                              filters=1,
                                              kernel_size=8,
                                              strides=4,
                                              padding='same')
        
#         outputs = wtf.add(deconv2, inputs)
        outputs = deconv2

#         wtf.summary.image('input', inputs, max_outputs=200)
#         wtf.summary.image('outputs', outputs, max_outputs=200)
#         wtf.summary.image('labels', labels, max_outputs=200)
        
        # Adds concatenated inputs and outputs to be displayed in TensorBoard
        disp = wtf.concat([inputs, labels, outputs], 1)
        wtf.summary.image('concat', disp, max_outputs=200)
    
        predictions = {'results': outputs}
    
        if mode == wtf.estimator.ModeKeys.PREDICT:
            export_outputs = {
                'predictions': wtf.estimator.export.PredictOutput(outputs)
            }
            return wtf.estimator.EstimatorSpec(
                mode=mode,
                predictions=outputs
            )
    
        # loss = wtf.losses.mean_squared_error(labels, outputs)
        loss = wtf.reduce_sum(wtf.abs(labels - outputs), axis=[1,2,3])
        loss = wtf.reduce_mean(loss)
        
        global_step = wtf.train.get_global_step()
        lr = wtf.train.exponential_decay(params.learning_rate, global_step, 1000, 0.96, staircase=True)
        
        wtf.summary.scalar('learning_rate', lr)
        
        optimizer = wtf.train.AdamOptimizer(
            learning_rate=lr,
            beta1=params.beta1,
            beta2=params.beta2
        )
    
        update_ops = wtf.get_collection(wtf.GraphKeys.UPDATE_OPS)
        with wtf.control_dependencies(update_ops):
            train_op = optimizer.minimize(
                loss=loss, global_step=wtf.train.get_global_step())
    
        eval_metric_ops = {
            "rmse": wtf.metrics.root_mean_squared_error(labels, outputs),
            "mae": wtf.metrics.mean_absolute_error(labels, outputs),
            "mse": wtf.metrics.mean_squared_error(labels, outputs)
        }
        
        eval_summary_hook = wtf.train.SummarySaverHook(
                                save_steps=1,
                                output_dir= "summary/eval/" + MODEL_NAME,
                                scaffold=wtf.train.Scaffold(summary_op=wtf.summary.merge_all()))
        
        evaluation_hooks=[]
        evaluation_hooks.append(eval_summary_hook)
    
        estimator_spec = wtf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=eval_metric_ops,
            evaluation_hooks=evaluation_hooks
        )
        
        return estimator_spec

In [5]:
def main(_):
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    
    wtf.logging.set_verbosity(wtf.logging.INFO)

    hparams = wtf.contrib.training.HParams(
        #num_epochs=1,
        #batch_size = 80,
        #forget_bias=1.0,
        learning_rate=0.00001,
        beta1=0.9,
        beta2=0.999
    )
    
    session_config = wtf.ConfigProto()
    session_config.gpu_options.allow_growth = True
    session_config.allow_soft_placement = True
    
    run_config = wtf.estimator.RunConfig(
        log_step_count_steps=10,
        save_summary_steps=10,
        tf_random_seed=19830610,
        model_dir=os.path.join('summary', 'train', MODEL_NAME),
        session_config=session_config
    )
    
    estimator = wtf.estimator.Estimator(
        model_fn=model_fn, params=hparams, config=run_config)
    
#     train_dataset = du.create_dataset('train', train_data)
#     train_dataset = du.create_dataset_fcn8('train', train_data)
    train_dataset = du.create_dataset_diff('val', val_data)
    train_dataset = train_dataset.shuffle(2000)
    train_dataset = train_dataset.batch(BATCH_SIZE)
    train_dataset = train_dataset.repeat(EPOCHS)
    
    val_dataset = du.create_dataset_diff('val', val_data)
    val_dataset = val_dataset.batch(1)
    
    def train_input_fn():
        train_iterator = train_dataset.make_one_shot_iterator()
        features, labels = train_iterator.get_next()
        return features, labels
    
    def val_input_fn():
        val_iterator = val_dataset.make_one_shot_iterator()
        features, labels = val_iterator.get_next()
        return features, labels
    
    estimator.train(input_fn=train_input_fn, max_steps=None)
    estimator.evaluate(input_fn=val_input_fn)

In [ ]:
if __name__ == '__main__':
    wtf.app.run()

Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Using config: {'_model_dir': 'summary/train/diff-001', '_tf_random_seed': 19830610, '_save_summary_steps': 10, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  allow_growth: true
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd95b6ba898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving chec

INFO:tensorflow:global_step/sec: 1.82005
INFO:tensorflow:global_step/sec: 1.82284
INFO:tensorflow:global_step/sec: 1.82246
INFO:tensorflow:global_step/sec: 1.82236
INFO:tensorflow:global_step/sec: 1.82114
INFO:tensorflow:loss = 280.59006, step = 1500 (54.841 sec)
INFO:tensorflow:global_step/sec: 1.82525
INFO:tensorflow:global_step/sec: 1.824
INFO:tensorflow:global_step/sec: 1.82167
INFO:tensorflow:global_step/sec: 1.83097
INFO:tensorflow:global_step/sec: 1.82757
INFO:tensorflow:global_step/sec: 1.82128
INFO:tensorflow:global_step/sec: 1.82377
INFO:tensorflow:global_step/sec: 1.82145
INFO:tensorflow:global_step/sec: 1.82182
INFO:tensorflow:global_step/sec: 1.77393
INFO:tensorflow:loss = 297.69516, step = 1600 (54.975 sec)
INFO:tensorflow:global_step/sec: 1.76321
INFO:tensorflow:global_step/sec: 1.74832
INFO:tensorflow:global_step/sec: 1.7487
INFO:tensorflow:global_step/sec: 1.745
INFO:tensorflow:global_step/sec: 1.74549
INFO:tensorflow:global_step/sec: 1.7728
INFO:tensorflow:global_step

INFO:tensorflow:Loss for final step: 521.2726.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-03-07:25:15
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from summary/train/diff-001/model.ckpt-3188
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
